#  Collections Data to Linked Art - Cleveland Museum of Art - Simplified

This notebook offers a simplified transformation of Cultural Heritage collections data to Linked Art JSON-LD


## Import What We Need for Notebook
The first step is to import Python code libraries that we need for the notebook. It's necessary to run this cell before calling on the Python libraries later on.

See the [Transform Introduction](01-00-Transform-Intro.ipynb) for a description of the Python libraries used

In [36]:
try:
    import IPython
except:
    !pip install IPython
    import IPython   
    
from IPython.display import display,IFrame,HTML
 
try:
    import json
except:
    !pip install json
    import json 
  
try:
    import csv
except:
    !pip install csv
    import csv
    
import os

import cromulent
from cromulent.model import factory
    
from lib import linkedart as la 

try:
    import pandas as pd
except:
    !pip install pandas
    import pandas as pd
    
try:
    from ipy_table import *
except:
    !pip install ipy_table
    from ipy_table import *
    

# display python dictionary in HTML table
# adapated from https://bit.ly/3dVDy7v
def dict2table(d):

    # lists
    temp = []
    dictList = []

    # convert the dictionary to a list
    for key, value in d.items():
        temp = [key,value]
        dictList.append(temp)

    # create table with make_table
    make_table(dictList)
    
    # apply some styles to the table after it is created
    set_column_style(0, width='100', bold=True, color='hsla(225, 80%, 94%, 1)',align='left')
    set_column_style(1, width='100',align='left')

    # render the table
    return render()


## Input Data
- The input data have been sourced from the Cleveland Museum of Art - https://www.clevelandart.org/art/collection/search
- data are in CSV format
- The data are available from https://github.com/ClevelandMuseumArt/openaccess 

The table below is a view of the first line of the input data file.

In [37]:
file = './data/cma/input/data.csv'
mpg = pd.read_csv(file,nrows=1,low_memory=False)
mpg.head()

,id,accession_number,share_license_status,tombstone,current_location,title,title_in_original_language,series,series_in_original_language,creation_date,...,digital_description,wall_description,external_resources,citations,catalogue_raisonne,url,image_web,image_print,image_full,updated_at
0,74539,2015.449,CC0,"A Miller's Carriage, c. 1895. Albert-Charles L...",NaN,A Miller's Carriage,NaN,NaN,NaN,c. 1895,...,NaN,NaN,"{'wikidata': [], 'internet_archive': ['https:/...",NaN,NaN,https://clevelandart.org/art/2015.449,https://openaccess-cdn.clevelandart.org/2015.4...,https://openaccess-cdn.clevelandart.org/2015.4...,https://openaccess-cdn.clevelandart.org/2015.4...,2021-06-29 06:35:50.572000


## Transform CSV to Linked Art JSON-LD

We now open the file read to transform. 

There is also an initial step to remove Byte Order Mark (BOM) characters from the input file. For further information - https://en.wikipedia.org/wiki/Byte_order_mark

In [38]:
#remove BOM
s = open(file, mode='r', encoding='utf-8-sig').read()
open(file, mode='w', encoding='utf-8').write(s)

80557121

### Using Python's `csv` module

The Python `csv` module (including csv.DictReader) will be used to read the CSV file and create an object that maps the information in each row to a dict whose keys are fieldnames. For further information - https://docs.python.org/3/library/csv.html


In [39]:
allObjects = csv.DictReader(open(file, mode='r',encoding='utf-8'))

### Convert CSV to Python dictionary 
An example record in the `dict` object is shown below. You'll note that the fields names are now dictionary keys.

In [40]:

for obj in allObjects:
    print(json.dumps(obj,indent=2))
    break  


{
  "id": "74539",
  "accession_number": "2015.449",
  "share_license_status": "CC0",
  "tombstone": "A Miller's Carriage, c. 1895. Albert-Charles Lebourg (French, 1849-1928). Black and white chalk with stumping ; sheet: 33.2 x 49.7 cm (13 1/16 x 19 9/16 in.). The Cleveland Museum of Art, Bequest of Muriel Butkin 2015.449",
  "current_location": "",
  "title": "A Miller's Carriage",
  "title_in_original_language": "",
  "series": "",
  "series_in_original_language": "",
  "creation_date": "c. 1895",
  "creation_date_earliest": "1890",
  "creation_date_latest": "1900",
  "creators": "Albert-Charles Lebourg (French, 1849-1928), artist",
  "culture": "France, 19th-20th century",
  "technique": "Black and white chalk with stumping ",
  "support_materials": "gray laid paper",
  "department": "Drawings",
  "collection": "DR - French",
  "type": "Drawing",
  "measurements": "Sheet: 33.2 x 49.7 cm (13 1/16 x 19 9/16 in.)",
  "state_of_the_work": "",
  "edition_of_the_work": "",
  "creditline":

### Create field mapping

The next step is to map entities in the collections data to their corresponding representation in Linked Art.

To achieve this: 
- a Python dictionary will be created
- field names in the input data mapped to variable names that are used in a mapping function that transforms the input data to Linked Art.


- Keys == input data field names
- Values == variables used in mapping functions

The `mapp` dictionary is shown below and also shown in table format beneath.

In [41]:
mapp =  {
    "id":"id",
    "accession_number":"accession_number",
    "accession_date": "",
    "classification" : "type",
    "title": "title",
    "alt_title": "title_in_original_language",
    "notes": "tombstone",
    "date_created":"creation_date",
    "date_created_earliest": "creation_date_earliest",
    "date_created_latest": "creation_date_latest",
    "created_period":"culture",
    "created_dynasty":"",
    "created_inscriptions":"inscriptions",
    "created_notes": "fun_fact",
    "creator":"creators",
    "physical_medium": "Medium",
    "physical_style": "",
    "physical_technique": "technique",
    "physical_description": "",
    "physical_dimensions": "measurements",
    "created_provenance": "provenance" ,
    "credit_line": "creditline",
    "collection" : "department",
    "current_status" : "current_location",
    "current_location": "current_location",
    "homepage": "url"
    }

In [42]:
dict2table(mapp)

id,id
accession_number,accession_number
accession_date,
classification,type
title,title
alt_title,title_in_original_language
notes,tombstone
date_created,creation_date
date_created_earliest,creation_date_earliest
date_created_latest,creation_date_latest
created_period,culture


### Create object property dictionary

The `mapp` dictionary will now be used as an input, together with the input data, to a function `createObjProp` that will create the Linked Art JSON-LD.

`createObjProp` created a Python dictionary that:
- has key/value pairs that a combination of variable names as keys and collections data values
- is used in `createObjDesc` to create Linked Art JSON-LD


If a straightforward mapping key:value pair mapping isn't possible, some custom code is used, e.g. for the `creators` mapping.

In [47]:
def createObjProp(obj,mapp):
    objProp = {}
    csv_keys = list(obj.keys())
    for key in csv_keys:
        for prop in mapp:
            if key == mapp[prop]:
                if prop == "creators":
                    objProp[prop] = {"name": obj[key], "id":"","role":"artist"}
                else:
                    objProp[prop] = obj[key]
    
    objProp["current_owner"] = {"name":"Cleveland Museum of Art",
                                "location":"Cleveland,Ohio",
                                "type": "http://vocab.getty.edu/aat/300312281" ,
                                "type_label": ""}
    return objProp

Iterate through the Python dictionary created from the input data with a for loop
- create Python dictionary with call to function `createObjProp` 
 - key/value pairs are are variable names/input data values 
 - used in `createObjDesc` to create Linked Art JSON-LD
 
The following table shows you an example of `objProp` Python dictionary

In [48]:
objProp={}

for obj in allObjects:
    objProp = createObjProp(obj,mapp)
    break
    
dict2table(objProp)

id,74554
accession_number,2015.447
notes,"Un Borreau (An Executioner), c. 1848. Auguste Raffet (French, 1804-1860). Watercolor, gouache and black chalk ; sheet: 32.8 x 25 cm (12 15/16 x 9 13/16 in.). The Cleveland Museum of Art, Bequest of Muriel Butkin 2015.447"
current_status,
current_location,
title,Un Borreau (An Executioner)
alt_title,
date_created,c. 1848
date_created_earliest,1843
date_created_latest,1853
creator,"Auguste Raffet (French, 1804-1860), artist"


The following code is usually in a separate file, but shown here for illustration.

It makes use of `cromulent` - written by Rob Sanderson - further information at https://github.com/linked-art/crom



In [ ]:
import cromulent

from cromulent.model import factory, Actor, Production, BeginningOfExistence, EndOfExistence, TimeSpan, Place
from cromulent.model import InformationObject, Phase, VisualItem 
from cromulent.vocab import Painting, Drawing,Miniature,add_art_setter, PrimaryName, Name, CollectionSet, instances, Sculpture 
from cromulent.vocab import aat_culture_mapping, AccessionNumber, Height, Width, SupportPart, Gallery, MuseumPlace 
from cromulent.vocab import BottomPart, Description, RightsStatement, MuseumOrg, Purchase
from cromulent.vocab import Furniture, Mosaic, Photograph, Coin, Vessel, Graphic, Enamel, Embroidery, PhotographPrint
from cromulent.vocab import PhotographAlbum, PhotographBook, PhotographColor, PhotographBW, Negative, Map, Clothing, Furniture
from cromulent.vocab import Sample, Architecture, Armor, Book, DecArts, Implement, Jewelry, Manuscript, SiteInstallation, Text, Print
from cromulent.vocab import TimeBasedMedia, Page, Folio, Folder, Box, Envelope, Binder, Case, FlatfileCabinet
from cromulent.vocab import HumanMadeObject,Tapestry,LocalNumber
from cromulent.vocab import Type,Set
from cromulent.vocab import TimeSpan, Actor, Group, Acquisition, Place
from cromulent.vocab import Production, TimeSpan, Actor
from cromulent.vocab import LinguisticObject,DigitalObject, DigitalService



"""
Dictionary of Object Types defined in Linked Art
"""


objTypes = {
"Painting": Painting(),
"Sculpture": Sculpture(),
"Drawing": Drawing(),
"Miniature": Miniature(),
"Tapestry": Tapestry(),
"Furniture": Furniture(),
"Furnishings": DecArts(),
"Mosaic": Mosaic(),
"Photograph": Photograph(),
"Coin": Coin(),
"Vessel": Vessel(),
"Graphic": Graphic(),
"Enamel": Enamel(),
"Embroidery": Embroidery(),
"PhotographPrint": PhotographPrint(),
"PhotographAlbum": PhotographAlbum(),
"PhotographBook": PhotographBook(),
"PhotographColor": PhotographColor(),
"PhotographBW": PhotographBW(),
"Negative": Negative(),
"Map": Map(),
"Clothing": Clothing(),
"Sample": Sample(),
"Architecture": Architecture(),
"Armor": Armor(),
"Book": Book(),
"DecArts": DecArts(),
"Implement": Implement(),
"Jewelry": Jewelry(),
"Manuscript": Manuscript(),
"SiteInstallation": SiteInstallation(),
"Text": Text(),
"Print": Print(),
"TimeBasedMedia": TimeBasedMedia(),
"Page": Page(),
"Folio": Folio(),
"Folder": Folder(),
"Box": Box(),
"Envelope": Envelope(),
"Binder": Binder(),
"Case": Case(),
"FlatfileCabinet": FlatfileCabinet()
}

def createObjProp(obj,docProp):
    objProp = {"creator":[]}

    for prop in obj["atom"]: 
        propName = prop["@name"]
        propValue = ""
        if "#text" in prop:
            propValue = prop["#text"]
        if propName in list(docProp.keys()):
            propId = docProp[propName]
            objProp[propId] = propValue
 

    # alternative titles

    for table in obj["table"]:
        if table["@name"] == "AltTitles": 
            alt_title = table["tuple"]["atom"]["#text"]
            objProp["alt_title"] = alt_title
        

        if table["@name"] == "Creator1": 
            crerole = creid = crename = ""
            if "atom" in table["tuple"]:
                for atom in table["tuple"]["atom"]:
                    if atom["@name"] == "CreRole":
                        crerole = atom["#text"]
                    if atom["@name"] == "irn":
                        creid = atom["#text"]
                    if atom["@name"] == "SummaryData":
                        crename = atom["#text"]
            creator = {"id":creid,"name":crename,"role":crerole}
            objProp["creator"].append(creator)
    
    return objProp

def objPrimaryname(objProp,object_uri):
    primaryname = None
    title = objProp["title"]
    id = str(objProp["id"])
    primaryname = PrimaryName( object_uri + "/primary-name",
                                value=title)
    return primaryname

def objAlternatename(objProp,object_uri):
    alternateName = None
    if "alt_title" in objProp:
        alt_title = objProp["alt_title"]
        alternatename = AlternateName(object_uri +  "/alternate-name",value=alt_title)
    return alternateName

def objHomepage(objProp,object_uri):

    homepage = None
    homepageId = objProp["homepage"]
    if homepageId != "":
        homepage = LinguisticObject(homepageId, label="Homepage for the Object")
        homepage.classified_as = Type("http://vocab/getty.edu/aat/300264578", label="Web pages (documents)")
        homepage.classified_as = Type("http://vocab.getty.edu/aat/300266277", label="home pages")
        homepage.format = "text/html"
    return homepage


def objProvenance(objProp,object_uri):

    prov = None
    if "created_provenance" in objProp:
        provenance = objProp["created_provenance"]
        if provenance !="":
            prov = LinguisticObject(object_uri + "/provenance-statement", 
                            value=provenance,
                            label="Provenance Statement about the Object"
                           )
            prov.classified_as = Type("http://vocab.getty.edu/aat/300055863", label="provenance (history of ownership)")
            prov.classified_as = Type("http://vocab.getty.edu/aat/300418049", label="brief texts")
        return prov


def objAccession(objProp, object_uri):
    accession = None
    accession_number = objProp["accession_number"]
    if accession_number != "":
        accession = AccessionNumber(accession_number,value=accession_number)
    return accession

def objLocalnumber(objProp,object_uri):
    localnumber = None
    id = str(objProp["id"])
    if id != "":
        localnumber = LocalNumber(id,value=id)
    return localnumber

def objCollection(objProp,object_uri):
    coll = None
    if "collection" in objProp:
        collection = objProp["collection"]
        coll = Set(object_uri +"/collection/" + collection, 
                   label= collection)
        coll.classified_as = Type("http://vocab.getty.edu/aat/300025976", 
                              label="collections (object groupings)")
    return coll



def objCredit(objProp,object_uri):
    credit = None
    propCredit = "credit_line"
    
    if propCredit in objProp:
        credit_line = objProp[propCredit]
        if credit_line != "":
            credit = LinguisticObject(object_uri + "/credit-line", 
                            value=credit_line,
                            label="Credit Line for the Object"
                           )
            credit.classified_as = Type("http://vocab.getty.edu/aat/300026687", label="acknowledgements")
            credit.classified_as = Type("http://vocab.getty.edu/aat/300418049", label="brief texts")
    return credit



def objProduction(objProp,object_uri):

    prod = None

    date_created = "date_created"
    created_earliest = "date_created_earliest"
    created_latest = "date_created_latest"

    if date_created in objProp:
        prod = Production(object_uri + "/production", label="Production of the Object")    
    
        labelTimespan = "date unknown"
        if objProp[date_created] != "":
            labelTimespan = objProp[date_created]
   
        timespan = TimeSpan(object_uri + "/production/timespan", label=labelTimespan)
        
        if created_earliest in objProp:
            timespan.begin_of_the_begin = objProp[created_earliest]
        if created_latest in objProp:
            timespan.end_of_the_end = objProp[created_latest]
    
        prod.timespan = timespan

        propCreator = "creator" 
        if propCreator in objProp:
            for creator in objProp[propCreator]:
                id = label = ""
                if "id" in creator:
                    id = creator["id"]
                if "name" in creator:
                    label = creator["name"]
                actor = Actor(id,label)

                prod.carried_out_by = actor
            
    return prod

def objCurrentowner(objProp,object_uri):
    current_owner = None
    if  "current_owner" in objProp and objProp["current_owner"]["name"] != "":
        cowner = objProp["current_owner"]["name"]
        current_owner = Group( "http://vocab.getty.edu/ulan/500300517",label=cowner)
        current_owner.classified_as = Type( "http://vocab.getty.edu/aat/300312281",label="museums (institutions)")
        
        
        acquisition = objAcquisition(objProp,object_uri)
        if acquisition is not None:
            current_owner.acquired_title_through = acquisition 
    return current_owner 

def objAcquisition(objProp,object_uri):
    acquisition = None
    if "accession_date" in objProp and objProp["accession_date"] != "":
        acquisition = Acquisition(object_uri + "/IMA-acquisition", label = "Acquisition of the Object")
        acquisition.classified_as = Type("http://vocab.getty.edu/aat/300157782",
                                         label="acquisition (collections management)")
        
        if "name_location" in objProp["current_owner"]:
            acquisition.took_place_at = Place("http://vocab.getty.edu/tgn/7012924", 
                                              label=objProp["current_owner"]["name_location"]) 
        acquisition.timespan = objAcquisitionTimespan(object_uri,objProp["accession_date"])
    return acquisition

def objAcquisitionTimespan(object_uri,accession_date):
    timespan = None
    end = begin = ""
    accession_date = str(accession_date)
    if len(accession_date) == 4:
        begin = accession_date + "-01-01T00:00:00.000Z"
        end = accession_date + "-12-31T00:00:00.000Z"
        
    elif len(accession_date) == 8:
        begin = accession_date + "01T00:00:00.000Z"
        end = accession_date 
        if '-02-' in accession_date:
            end = end + "28"
        if ('-01-','-03-','-05-','-07-','-08-','-09-','-10-','-12-') in accession_date:
            end = end + "31"
        if ('-04-','-06-','-09-','-11-'):
            end = end + "30"      
        end = end + "T00:00:00"
         
    elif len(accession_date) == 10:
        begin = accession_date + "T00:00:00.000Z"
        end = accession_date + "T00:00:00.000Z"    
    else:
        begin = end = ""
        
    timespan = TimeSpan(object_uri + "/IMA-acquisition/timespan", label=accession_date)
        
    if begin != "":
        timespan.begin_of_the_begin = begin
    if end != "":
        timespan.end_of_the_end = end
    return timespan
    

def objCustody(objProp,object_uri):
    
    custody = None

    if "current_status" in objProp and objProp["current_status"] != "" :
        current_status = objProp["current_status"]
        current_owner = checkCurrentOwner(current_status)
    
        if current_owner == False:
            name = objProp["current_owner"]["name"]
            type = objProp["current_owner"]["type"]
            label = objProp["current_owner"]["type_label"]
            
            custody = Group(label=name)
            custody.classified_as = Type(type, 
                                     label=label)
    return custody


def objDigitalObject(objProp,object_uri):
    vi = None
    url = objProp["image_url"]
    vi = VisualItem(url, label="Digital image of artwork")

    do = DigitalObject(url,label="Image of artwork")
    do.classified_as = Type("http://vocab.getty.edu/aat/300215302","Digital Image")
    do.access_point = DigitalObject(url)

    if "iiif" in url.lower():
        ds = DigitalService()
        ds.access_point = DigitalObject(url)
        ds.conforms_to = InformationObject("http://iiif.io/api/image")
        do.digitally_available_via = ds
        
    vi.digitally_shown_by = do

    return vi
    

def createObjDesc(objProp,objTypes,object_uri):
    objLA = None
    objLA = HumanMadeObject() # linked art object

    for otype in objTypes:
        if otype in objProp["classification"]:
            objLA = objTypes[otype]        
            break
            
    objLA.id = object_uri
    objLA._label =  objProp["title"]
    
    # DIGITAL OBJECT
    if "image_url" in objProp and objProp["image_url"] != "":
        dig = objDigitalObject(objProp,object_uri)
        if dig is not None:
            objLA.representation = []
            objLA.representation.append(dig)


    # IDENTIFIED_BY 
    accession = objAccession(objProp,object_uri)
    localnumber = objLocalnumber(objProp,object_uri)
    primaryname = objPrimaryname(objProp,object_uri)
    
    listIds = (accession,localnumber,primaryname)
    identified_by = False
    for id in listIds:
        if id is not None:
            identified_by = True
            break
    if identified_by == True:
        objLA.identified_by = []
        
        for id in listIds:
            if id is not None:
                objLA.identified_by.append(id)
    
    # REFERRED_TO_BY
    objLA.referred_to_by = None
    prov = objProvenance(objProp,object_uri)
    credit = objCredit(objProp,object_uri)
    referred_to_by = False
    if prov is not None or credit is not None:
        referred_to_by = True
    if referred_to_by == True:
        objLA.referred_to_by = []
    if prov is not None:
        objLA.referred_to_by.append(prov) # provenance 
    if credit is not None:
        objLA.referred_to_by.append(credit) # credit line
    
    # SUBJECT_OF 
    objLA.subject_of = None 
    homepage = None  
    homepage = objHomepage(objProp,object_uri)
    if homepage is not None:
        objLA.subject_of = homepage # home page
        
    # PRODUCED_BY
    objLA.produced_by = None 
    if "creator" in objProp:
        prod = None
       # prod = objProduction(objProp,object_uri)
      #  objLA.produced_by = None
        if prod is not None:
            objLA.produced_by = prod # production

    # MEMBER_OF
    objLA.member_of = None 
    if "collection" in objProp:
        coll = None
        coll = objCollection(objProp,object_uri)
        if coll is not None:
            objLA.member_of = coll # collection

    # CURRENT_KEEPER
    objLA.current_owner = None 
    custody = None
    custody = objCustody(objProp,object_uri)
    if custody is not None:
        objLA.current_owner = custody
    
    # CURRENT_OWNER
    if "current_owner" in objProp and objProp["current_owner"] != "":
        current_owner = objCurrentowner(objProp,object_uri)
        if current_owner is not None:
            objLA.current_owner = current_owner
          
    return objLA

def checkCurrentOwner(current_status):
    currentowner = False
    if current_status != "":
        checkObjStatus = ('Accessioned','Partial Accession')
        for status in checkObjStatus:
            if status == current_status:
                currentowner = True
            if 'IMA-Owned' in current_status:
                currentowner = True
    return currentowner


### Generate Linked Art JSON-LD and write to file

Final part of transformation is to call `createObjDesc` using 
 - `objProp`
 - `object_uri`
 - `objTypes` - different types of artworks defined in `cromulent` 
 
The will return the collections data record transformed to Linked Art JSON-LD that can be written to file.

In [55]:
#  baseURI for JSON-LD document
baseURI = "https://clevelandart.org/art/"

id = obj[mapp.get("id")]
object_uri = baseURI + id
    
objLA = None
objLA = la.createObjDesc(objProp,la.objTypes,object_uri)
    
# write to file 
text_file = open("./data/cma/output/json/all/" + id + ".json", "wt")
n = text_file.write(factory.toString(objLA, compact=False))
text_file.close()
  

### Visualise JSON-LD

Linked Art JSON-LD - Visualise with JSON-LD Playground
The resulting Linked Art JSON-LD is shown below.

You can visualise this with a copy+paste of the JSON-LD and a visit to https://json-ld.org/playground/ - there is a visualisation option/tabl

In [56]:
print(factory.toString(objLA, compact=False))

{
  "@context": "https://linked.art/ns/v1/linked-art.json",
  "id": "https://clevelandart.org/art/74554",
  "type": "HumanMadeObject",
  "_label": "Un Borreau (An Executioner)",
  "classified_as": [
    {
      "id": "http://vocab.getty.edu/aat/300033973",
      "type": "Type",
      "_label": "Drawing",
      "classified_as": [
        {
          "id": "http://vocab.getty.edu/aat/300435443",
          "type": "Type",
          "_label": "Type of Work"
        }
      ]
    }
  ],
  "identified_by": [
    {
      "id": "http://lod.example.org/museum/Identifier/2015.447",
      "type": "Identifier",
      "classified_as": [
        {
          "id": "http://vocab.getty.edu/aat/300312355",
          "type": "Type",
          "_label": "Accession Number"
        }
      ],
      "content": "2015.447"
    },
    {
      "id": "http://lod.example.org/museum/Identifier/74554",
      "type": "Identifier",
      "classified_as": [
        {
          "id": "http://vocab.getty.edu/aat/30040462

## Next steps

View other notebooks
- transformation
- reconcilation
- data visualisation